In [1]:
import sys
print(sys.executable)


c:\projects\retail-analytics-dashboard\retail_env\Scripts\python.exe


In [2]:
# Cell 1: Verify data exists
import os
import pandas as pd

file_path = '../data/processed/clean_data.csv'

if os.path.exists(file_path):
    print(" Data file found!")
    df = pd.read_csv(file_path)
    print(f" Loaded {len(df):,} rows")
else:
    print(" Data file not found. Run scripts/01_data_ingestion.py first!")

 Data file found!
 Loaded 397,884 rows


In [3]:
# Cell 1: Setup
"""
Exploratory Data Analysis - Retail Dataset
Goal: Understand data patterns, identify insights for dashboard
"""

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

# Styling
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports complete")

✓ Imports complete


In [4]:
# Cell 2: Load Data
df = pd.read_csv('../data/processed/clean_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

Dataset shape: (397884, 8)

Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

First few rows:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
# Cell 3: Quick Stats
print("=== Dataset Overview ===\n")
print(df.info())
print("\n=== Statistical Summary ===\n")
print(df.describe())

=== Dataset Overview ===

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397884 entries, 0 to 397883
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    397884 non-null  int64  
 1   StockCode    397884 non-null  object 
 2   Description  397884 non-null  object 
 3   Quantity     397884 non-null  int64  
 4   InvoiceDate  397884 non-null  object 
 5   UnitPrice    397884 non-null  float64
 6   CustomerID   397884 non-null  float64
 7   Country      397884 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 24.3+ MB
None

=== Statistical Summary ===

           InvoiceNo       Quantity      UnitPrice     CustomerID
count  397884.000000  397884.000000  397884.000000  397884.000000
mean   560616.934451      12.988238       3.116488   15294.423453
std     13106.117773     179.331775      22.097877    1713.141560
min    536365.000000       1.000000       0.001000   12346.000000
25%   